In [1]:
# ==============================================================================
# CÉLULA 1: Configuração e Importações
# ==============================================================================
import pandas as pd
import numpy as np
import os
import psycopg2
from psycopg2.extras import execute_values

# Configurações de exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Caminhos dos Arquivos (Ajuste conforme onde você salvou o CSV do Kaggle)
INPUT_FILE = '../Data Layer/raw/data_raw.csv'
OUTPUT_DIR = 'Data Layer/silver/diabetes'
OUTPUT_DIR = '../Data Layer/silver/data'
OUTPUT_FILE_CSV = 'diabetes_silver.csv'

# Configuração do Banco de Dados
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'diabetes_health', # Crie este banco no Postgres antes
    'user': 'postgres',
    'password': 'postgres' # Sua senha
}

In [2]:
# ==============================================================================
# CÉLULA 2: Funções Auxiliares de Mapeamento (CORRIGIDA)
# ==============================================================================

def map_binary_yes_no(value):
    """Transforma 0/1 em Não/Sim."""
    if pd.isna(value): return 'Desconhecido'
    # Correção: float(value) lida com '0.0' e '1.0'
    if int(float(value)) == 1: return 'Sim'
    return 'Não'

def map_diabetes_status(value):
    """0 = Não, 1 = Pré, 2 = Diabetes"""
    if pd.isna(value): return 'Desconhecido'
    val = int(float(value)) # Correção aqui também
    if val == 0: return 'Não'
    elif val == 1: return 'Pré-diabetes'
    elif val == 2: return 'Diabetes'
    return 'Desconhecido'

def map_sex(value):
    if pd.isna(value): return 'Desconhecido'
    # Correção aqui também
    return 'Masculino' if int(float(value)) == 1 else 'Feminino'

def map_age_group(value):
    if pd.isna(value): return 'Desconhecido'
    val = int(float(value)) # Correção
    mapping = {
        1: '18-24', 2: '25-29', 3: '30-34', 4: '35-39', 5: '40-44',
        6: '45-49', 7: '50-54', 8: '55-59', 9: '60-64', 10: '65-69',
        11: '70-74', 12: '75-79', 13: '80+'
    }
    return mapping.get(val, 'Desconhecido')

def map_education(value):
    if pd.isna(value): return 'Desconhecido'
    val = int(float(value)) # Correção
    mapping = {
        1: 'Nunca frequentou escola', 2: 'Fundamental incompleto',
        3: 'Fundamental completo', 4: 'Médio incompleto',
        5: 'Médio completo', 6: 'Superior completo'
    }
    return mapping.get(val, 'Desconhecido')

def map_general_health(value):
    if pd.isna(value): return 'Desconhecido'
    val = int(float(value)) # Correção
    mapping = {1: 'Excelente', 2: 'Muito Boa', 3: 'Boa', 4: 'Razoável', 5: 'Ruim'}
    return mapping.get(val, 'Desconhecido')

def classify_bmi(bmi):
    if pd.isna(bmi): return 'Desconhecido'
    # BMI já é float, não precisa converter para int, só garantir float
    bmi_val = float(bmi)
    if bmi_val < 18.5: return 'Abaixo do Peso'
    elif bmi_val < 25: return 'Peso Normal'
    elif bmi_val < 30: return 'Sobrepeso'
    else: return 'Obesidade'

In [3]:
# ==============================================================================
# CÉLULA 3: Carregamento e Limpeza Inicial (BLINDADA)
# ==============================================================================
print("Carregando dados Bronze...")
# Carrega ignorando erros de linha ruim (caso tenha sobrado alguma)
df = pd.read_csv(INPUT_FILE, on_bad_lines='skip') 

# --- VACINA CONTRA ESPAÇOS ---
# Remove espaços antes e depois dos nomes das colunas
# Ex: " Income " vira "Income"
df.columns = df.columns.str.strip()

print(f"Linhas carregadas: {len(df):,}")
print(f"Colunas encontradas: {list(df.columns)}")

# Verifica se as colunas essenciais estão lá
colunas_necessarias = ['Diabetes_012', 'CholCheck', 'Income']
for col in colunas_necessarias:
    if col not in df.columns:
        print(f"⚠️ ALERTA: Coluna '{col}' NÃO encontrada no CSV! Verifique o nome exato acima.")
    else:
        print(f"✅ Coluna '{col}' encontrada.")

# Criação de ID Único
df['patient_id'] = range(1, len(df) + 1)

# Remoção de Duplicatas
cols_data = [c for c in df.columns if c != 'patient_id']
df = df.drop_duplicates(subset=cols_data).reset_index(drop=True)

Carregando dados Bronze...
Linhas carregadas: 253,680
Colunas encontradas: ['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']
✅ Coluna 'Diabetes_012' encontrada.
✅ Coluna 'CholCheck' encontrada.
✅ Coluna 'Income' encontrada.


In [4]:
# ==============================================================================
# CÉLULA 4: Transformações (CORRIGIDA)
# ==============================================================================
print("Aplicando transformações...")

# 1. Mapeamento do Diabetes (Garante que a coluna source existe)
if 'Diabetes_012' in df.columns:
    df['diabetes_status'] = df['Diabetes_012'].apply(map_diabetes_status)
else:
    print("ERRO CRÍTICO: Não foi possível criar 'diabetes_status' pois 'Diabetes_012' sumiu.")

# 2. Colunas Binárias (0/1 -> Não/Sim)
# Adicionei verificação de existência
cols_sim_nao = [
    'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 
    'HeartDiseaseorAttack', 'PhysActivity', 'HvyAlcoholConsump', 
    'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 
    'Fruits', 'Veggies'
]

for col in cols_sim_nao:
    if col in df.columns:
        df[col] = df[col].apply(map_binary_yes_no)

# 3. Outros Mapeamentos
# Usamos .get() para evitar erro se a coluna não existir, mas o ideal é que existam
if 'Sex' in df.columns: df['sex_desc'] = df['Sex'].apply(map_sex)
if 'Age' in df.columns: df['age_group'] = df['Age'].apply(map_age_group)
if 'Education' in df.columns: df['education_level'] = df['Education'].apply(map_education)
if 'GenHlth' in df.columns: df['general_health'] = df['GenHlth'].apply(map_general_health)
if 'BMI' in df.columns: df['bmi_category'] = df['BMI'].apply(classify_bmi)

# 4. Score de Risco
def calculate_risk_score(row):
    score = 0
    # Usa .get para segurança
    if row.get('HighBP') == 'Sim': score += 1
    if row.get('HighChol') == 'Sim': score += 1
    if row.get('Smoker') == 'Sim': score += 1
    if row.get('bmi_category') == 'Obesidade': score += 1
    return score

df['risk_factors_count'] = df.apply(calculate_risk_score, axis=1)

print("Transformações concluídas.")

Aplicando transformações...
Transformações concluídas.


In [5]:
# ==============================================================================
# CÉLULA 5: Renomeação e Seleção Final
# ==============================================================================
rename_map = {
    'Diabetes_012': 'diabetes_code_raw',
    # As chaves aqui TEM que bater com os nomes limpos na Célula 3
    'CholCheck': 'cholesterol_check', 
    'Income': 'income_level_raw',
    'HighBP': 'high_bp',
    'HighChol': 'high_chol',
    'BMI': 'bmi',
    'Smoker': 'smoker',
    'Stroke': 'stroke',
    'HeartDiseaseorAttack': 'heart_disease_attack',
    'PhysActivity': 'physical_activity',
    'Fruits': 'eats_fruits',
    'Veggies': 'eats_veggies',
    'HvyAlcoholConsump': 'heavy_alcohol',
    'AnyHealthcare': 'has_healthcare',
    'NoDocbcCost': 'cant_afford_doctor',
    'GenHlth': 'gen_health_score',
    'MentHlth': 'mental_health_days',
    'PhysHlth': 'physical_health_days',
    'DiffWalk': 'diff_walking',
    'Sex': 'sex_raw',
    'Age': 'age_raw',
    'Education': 'education_raw'
}

df_silver = df.rename(columns=rename_map)

# Lista final obrigatória
final_columns = [
    'patient_id', 'diabetes_status', 'high_bp', 'high_chol', 'cholesterol_check',
    'bmi', 'bmi_category', 'smoker', 'stroke', 'heart_disease_attack',
    'physical_activity', 'eats_fruits', 'eats_veggies', 'heavy_alcohol',
    'has_healthcare', 'cant_afford_doctor', 'gen_health_score', 'general_health',
    'mental_health_days', 'physical_health_days', 'diff_walking',
    'sex_desc', 'age_group', 'education_level', 'income_level_raw', 'risk_factors_count'
]

# Verifica se sobrou alguma coluna faltando AGORA
missing = [c for c in final_columns if c not in df_silver.columns]
if missing:
    print(f"❌ AINDA FALTAM: {missing}")
    print("Colunas disponíveis:", df_silver.columns.tolist())
else:
    print("✅ Todas as 26 colunas estão presentes!")
    df_silver = df_silver[final_columns]

✅ Todas as 26 colunas estão presentes!


In [6]:
# ==============================================================================
# CÉLULA 6: Salvar CSV Silver
# ==============================================================================
print("Salvando arquivo Silver...")
os.makedirs(OUTPUT_DIR, exist_ok=True)
csv_path = os.path.join(OUTPUT_DIR, OUTPUT_FILE_CSV)
df_silver.to_csv(csv_path, index=False, encoding='utf-8')
print(f"Arquivo salvo em: {csv_path}")

Salvando arquivo Silver...
Arquivo salvo em: ../Data Layer/silver/data/diabetes_silver.csv


In [7]:
# Debug: Verificar quais colunas estão indo para o banco
cols_sql = [
    'patient_id', 'diabetes_status', 'high_bp', 'high_chol', 'cholesterol_check',
    'bmi', 'bmi_category', 'smoker', 'stroke', 'heart_disease_attack',
    'physical_activity', 'eats_fruits', 'eats_veggies', 'heavy_alcohol',
    'has_healthcare', 'cant_afford_doctor', 'gen_health_score', 'general_health',
    'mental_health_days', 'physical_health_days', 'diff_walking',
    'sex_desc', 'age_group', 'education_level', 'income_level_raw', 'risk_factors_count'
]

print(f"O SQL espera {len(cols_sql)} colunas.")
print(f"O DataFrame tem {len(df_silver.columns)} colunas.")

# Mostrar qual está faltando
missing = [col for col in cols_sql if col not in df_silver.columns]
print(f"Colunas FALTANDO no DataFrame: {missing}")

O SQL espera 26 colunas.
O DataFrame tem 26 colunas.
Colunas FALTANDO no DataFrame: []


In [8]:
# ==============================================================================
# CÉLULA 7: Carga no PostgreSQL (ATUALIZADA)
# ==============================================================================
create_table_sql = """
DROP TABLE IF EXISTS silver.diabetes_indicators CASCADE;
CREATE TABLE IF NOT EXISTS silver.diabetes_indicators (
    patient_id INT PRIMARY KEY,
    diabetes_status VARCHAR(20),  -- Mudou de has_diabetes para status
    high_bp VARCHAR(10),
    high_chol VARCHAR(10),
    cholesterol_check VARCHAR(10), -- Nova
    bmi FLOAT,
    bmi_category VARCHAR(20),
    smoker VARCHAR(10),
    stroke VARCHAR(10),
    heart_disease_attack VARCHAR(10),
    physical_activity VARCHAR(10),
    eats_fruits VARCHAR(10),     -- Mudou para varchar (Sim/Não)
    eats_veggies VARCHAR(10),    -- Mudou para varchar (Sim/Não)
    heavy_alcohol VARCHAR(10),
    has_healthcare VARCHAR(10),
    cant_afford_doctor VARCHAR(10),
    gen_health_score INT,
    general_health VARCHAR(20),
    mental_health_days INT,
    physical_health_days INT,
    diff_walking VARCHAR(10),
    sex_desc VARCHAR(15),
    age_group VARCHAR(20),
    education_level VARCHAR(50),
    income_level_raw INT,        -- Nova
    risk_factors_count INT
);
"""

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute("CREATE SCHEMA IF NOT EXISTS silver;")
    cur.execute(create_table_sql)
    conn.commit()
    print("Tabela recriada.")

    data_values = df_silver.where(pd.notnull(df_silver), None).values.tolist()

    insert_sql = """
    INSERT INTO silver.diabetes_indicators (
        patient_id, diabetes_status, high_bp, high_chol, cholesterol_check,
        bmi, bmi_category, smoker, stroke, heart_disease_attack,
        physical_activity, eats_fruits, eats_veggies, heavy_alcohol,
        has_healthcare, cant_afford_doctor, gen_health_score, general_health,
        mental_health_days, physical_health_days, diff_walking,
        sex_desc, age_group, education_level, income_level_raw, risk_factors_count
    ) VALUES %s
    """

    execute_values(cur, insert_sql, data_values)
    conn.commit()
    print(f"Sucesso! {len(data_values)} registros inseridos.")

except Exception as e:
    print(f"Erro: {e}")
    conn.rollback()
finally:
    if 'conn' in locals(): conn.close()

Tabela recriada.
Sucesso! 229781 registros inseridos.
